In [1]:
import fiftyone as fo
from ultralytics import YOLO
import os

dataset_name = "HBFMID_Manual_Load"
if dataset_name not in fo.list_datasets():
    print(f"Errore: Il dataset '{dataset_name}' non è stato trovato. Assicurati di averlo caricato con lo script precedente.")
    exit()

dataset = fo.load_dataset(dataset_name)
print(f"Dataset '{dataset.name}' caricato con {len(dataset)} immagini.")

export_dir = "HBFMID_YOLO_Export" 

if os.path.exists(export_dir):
    import shutil
    shutil.rmtree(export_dir)
    print(f"Cartella esistente '{export_dir}' eliminata.")
os.makedirs(export_dir, exist_ok=True)


print(f"\nEsportazione del dataset in formato YOLO nella directory: {export_dir}...")
dataset.export(
    export_dir=export_dir,
    dataset_type=fo.types.YOLOv5Dataset, 
    label_field="ground_truth",         
    splits={                             
        "train": "train",
        "validation": "validation",
        "test": "test"
    }
)
print("Esportazione completata!")

data_yaml_path = os.path.abspath(os.path.join(export_dir, 'dataset.yaml'))
print(f"File dataset.yaml per training: {data_yaml_path}")

print("\nVerifica della struttura delle cartelle esportate:")
for root, dirs, files in os.walk(export_dir):
    level = root.replace(export_dir, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 4 * (level + 1)
    for f in files:
        print(f'{subindent}{f}')

You are running the oldest supported major version of MongoDB. Please refer to https://deprecation.voxel51.com for deprecation notices. You can suppress this exception by setting your `database_validation` config parameter to `False`. See https://docs.voxel51.com/user_guide/config.html#configuring-a-mongodb-connection for more information
Dataset 'HBFMID_Manual_Load' caricato con 1539 immagini.
Cartella esistente 'HBFMID_YOLO_Export' eliminata.

Esportazione del dataset in formato YOLO nella directory: HBFMID_YOLO_Export...
Directory 'HBFMID_YOLO_Export' already exists; export will be merged with existing files
Ignoring unsupported parameter 'splits'
 100% |███████████████| 1539/1539 [15.3s elapsed, 0s remaining, 67.6 samples/s]       
Esportazione completata!
File dataset.yaml per training: C:\Users\sgarr\Desktop\progetto_Vision\HBFMID_YOLO_Export\dataset.yaml

Verifica della struttura delle cartelle esportate:
HBFMID_YOLO_Export/
    dataset.yaml
    images/
        val/
            

In [ ]:
import fiftyone as fo
from ultralytics import YOLO
import os
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Utilizzando il dispositivo: {device}")

model = YOLO('yolo11n.pt') 

data_yaml_path = os.path.abspath("HBFMID_YOLO_Export/dataset.yaml")

def freeze_layer(trainer):
    model = trainer.model
    num_freeze = 10
    print(f"Freezing {num_freeze} layers")
    freeze = [f'model.{x}.' for x in range(num_freeze)]   
    for k, v in model.named_parameters(): 
        v.requires_grad = True 
        if any(x in k for x in freeze): 
            print(f'freezing {k}') 
            v.requires_grad = False 
    print(f"{num_freeze} layers are freezed.")


model.add_callback("on_train_start", freeze_layer)

print(f"\nInizio del fine-tuning del modello YOLO usando il dataset: {data_yaml_path}")

results = model.train(
    data=data_yaml_path,
    epochs=50,      
    imgsz=640, 
    batch=16,
    name='yolov11n_hbfmid_finetune_tentativo_freezeLayers',
    device=device,
    patience=20
)

print("\nFine-tuning completato!")
print(f"I risultati del training sono salvati in: {model.trainer.save_dir}")

Utilizzando il dispositivo: cuda

Inizio del fine-tuning del modello YOLO usando il dataset: C:\Users\sgarr\Desktop\progetto_Vision\HBFMID_YOLO_Export\dataset.yaml
New https://pypi.org/project/ultralytics/8.3.158 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.155  Python-3.13.2 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\sgarr\Desktop\progetto_Vision\HBFMID_YOLO_Export\dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=

train: Scanning C:\Users\sgarr\Desktop\progetto_Vision\HBFMID_YOLO_Export\labels\train.cache... 1347 images, 3 backgrou


val: Fast image access  (ping: 0.00.0 ms, read: 462.0151.2 MB/s, size: 29.9 KB)


val: Scanning C:\Users\sgarr\Desktop\progetto_Vision\HBFMID_YOLO_Export\labels\val.cache... 1539 images, 3 backgrounds,


Plotting labels to runs\detect\yolov11n_hbfmid_finetune_tentativo_freezeLayers\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Freezing 10 layers
freezing model.0.conv.weight
freezing model.0.bn.weight
freezing model.0.bn.bias
freezing model.1.conv.weight
freezing model.1.bn.weight
freezing model.1.bn.bias
freezing model.2.cv1.conv.weight
freezing model.2.cv1.bn.weight
freezing model.2.cv1.bn.bias
freezing model.2.cv2.conv.weight
freezing model.2.cv2.bn.weight
freezing model.2.cv2.bn.bias
freezing model.2.m.0.cv1.conv.weight
freezing model.2.m.0.cv1.bn.weight
freezing model.2.m.0.cv1.bn.bias
freezing model.2.m.0.cv2.conv.weight
freezing model.2.m.0.cv2.bn.weight
freezing model.2.m.0.cv2.bn.bias
freezing model.3.conv.weight
freezing model.3.bn

...

                   all       1539       1771    0.00867      0.173    0.00626    0.00304



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      40/50      3.56G      1.389      1.137      1.398          6        640: 100%|██████████| 85/85 [00:19<00:00,  4.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 49/49 [00:18

                   all       1539       1771     0.0092      0.172    0.00691      0.003


Closing dataloader mosaic
